In [1]:
import pyranges as pr
import pandas as pd
import numpy as np
import h5py 
import os

from cerberus.cerberus import *
from cerberus.main import *


## convert transcriptome

In [2]:
# def add_triplets(t_df, ic, tss, tes):
#     """
#     Determines which tss, intron chain, and tes are used from a cerberus
#     reference are used in a specific gtf.

#     Parameters:
#         gtf (str): File path to gtf you want to assign triplets to
#         ic_file (str): File path to intron chain tsv (part of cerberus ref.)
#         tss_bed (str): File path to tss bed (part of cerberus ref.)
#         tes_bed (str): File path to tes bed (part of cerberus ref.)

#     Returns:
#         df (pandas DataFrame): File that maps each transcript from gtf to
#             a tss, intron chain, and tes given by the cerberus reference.
#             Also includes new transcript ids to refer to each transcript
#     """

#     ### intron chain annotation ###

#     # get unique intron chains from gtf
#     df = t_df.copy()
#     df = get_ic(df)
#     df.rename({'ic': 'Coordinates'},
#               axis=1, inplace=True)

#     # read in reference set of intron chains
#     ref = ic

#     # merge on intron chain, strand, chromosome, and gene id
#     df = df.merge(ref, how='left',
#                  on=['Chromosome', 'Strand',
#                      'Coordinates', 'gene_id'])

#     # formatting
#     df.rename({'Name': 'ic_id'}, axis=1, inplace=True)
#     df = df[['transcript_id', 'ic', 'ic_id']]

#     ### end annotation ###

#     for mode, ref in zip(['tss', 'tes'], [tss, tes]):

#         if mode == 'tss':
#             ends = t_df.features.tss()
#         elif mode == 'tes':
#             ends = t_df.features.tes()

#         # merge transcriptome ends with reference ends
#         ref = read_cerberus_ends(bed_file, mode=mode)
#         ref = pr.PyRanges(ref)
#         ends = ends.join(ref,
#                          strandedness='same',
#                          how='left').df
#         ends = ends.loc[ends.gene_id == ends.gene_id_b]

#         # formatting
#         ends.rename({'Name': '{}_id'.format(mode)}, axis=1, inplace=True)
#         ends = ends[['transcript_id', mode, '{}_id'.format(mode)]]

#         # add ends into map df
#         df = df.merge(ends, how='left', on='transcript_id')

#     ### creating map file ###

#     # get gene id / name and transcript name from original gtf
#     t_df = t_df.df
#     t_df = t_df.loc[t_df.Feature == 'transcript']
#     if 'gene_name' not in t_df.columns:
#         t_df['gene_name'] = t_df.transcript_name.str.split('-', n=1, expand=True)[0]
#     t_df = t_df[['gene_id', 'gene_name',
#                  'transcript_id', 'transcript_name']]
#     df = df.merge(t_df, how='left', on='transcript_id')

#     # create triplets and rename old ids
#     df.rename({'transcript_id': 'original_transcript_id',
#                'transcript_name': 'original_transcript_name'},
#                axis=1, inplace=True)
#     df['transcript_triplet'] = '['+df.tss.astype(str)+', '+\
#                                    df.ic.astype(str)+', '+\
#                                    df.tes.astype(str)+']'
#     df['transcript_id'] = df['gene_id']+' '+df.transcript_triplet
#     df['transcript_name'] = df['gene_name']+' '+df.transcript_triplet

#     return df

In [3]:
def assign_triplets(gtf_df, tss, ic, tes):
    """
    Determines which tss, intron chain, and tes are used from a cerberus
    reference are used in a specific gtf.

    Parameters:
        gtf (pyranges PyRanges): PyRanges GTF object for transcriptome to assign triplets to
        ic_file (pandas DataFrame): df of intron chains from cerberus ref.
        tss_bed (str): PyRanges obj of tsss from cerberus ref
        tes_bed (str): PyRanges obj of tess from cerberus ref

    Returns:
        df (pandas DataFrame): File that maps each transcript from gtf to
            a tss, intron chain, and tes given by the cerberus reference.
            Also includes new transcript ids to refer to each transcript
    """

    ### intron chain ###

    # get intron chains from input transcriptome
    df = gtf_df.copy()
    df = get_ic(df)
    df.rename({'ic': 'Coordinates'}, axis=1, inplace=True)

    # merge on intron chain, strand, chromosome, and gene id
    df = df.merge(ic, how='left',
                  on=['Chromosome', 'Strand',
                      'Coordinates', 'gene_id'])

    # formatting
    df.rename({'Name': 'ic_id'}, axis=1, inplace=True)
    df = df[['transcript_id', 'ic', 'ic_id']]


    ### ends ###
    for mode, ref in zip(['tss', 'tes'], [tss, tes]):
        if mode == 'tss':
            ends = gtf_df.features.tss()
        elif mode == 'tes':
            ends = gtf_df.features.tes()

        # limit to relevant columns
        ends = ends[['Chromosome', 'Start', 'End', 'Strand',
                     'gene_id', 'transcript_id']]

        # get only the relevant columns and deduplicate
        ends = ends.df
        t_ends = ends.copy(deep=True)
        ends.drop('transcript_id', axis=1, inplace=True)
        ends.drop_duplicates(inplace=True)
        ends = pr.PyRanges(ends)

        # find closest interval in ref
        ends = ends.nearest(ref, 
                            strandedness=None)

        # fix the ends with mismatching gene ids - this part can be slow :(
        ends = ends.df
        fix_ends = ends.loc[ends.gene_id != ends.gene_id_b]
        fix_ends = fix_ends[['Chromosome', 'Start', 'End', 'Strand',
                             'gene_id']]
        ends = ends.loc[ends.gene_id == ends.gene_id_b]

        for i, gid in enumerate(fix_ends.gene_id.unique().tolist()):
            gene_ends = fix_ends.loc[fix_ends.gene_id == gid].copy(deep=True)
            gene_ends = pr.PyRanges(gene_ends)
            gene_refs = ref.df.loc[ref.df.gene_id == gid].copy(deep=True)
            gene_refs = pr.PyRanges(gene_refs)
            gene_ends = gene_ends.nearest(ref, 
                                          strandedness=None)
            gene_ends = gene_ends.df
            ends = pd.concat([ends, gene_ends])

            if i % 100 == 0:
                print('Processed {} / {} genes'.format(i, len(fix_ends.gene_id.unique().tolist())))

        # merge back in to get transcript ids
        t_ends = t_ends.merge(ends, how='left',
                          on=['Chromosome', 'Start', 'End', 'Strand', 'gene_id'])

        # formatting
        t_ends.rename({'Name': '{}_id'.format(mode)}, axis=1, inplace=True)
        t_ends = t_ends[['transcript_id', '{}_id'.format(mode), mode]]

        # merge with ic ids
        df = df.merge(t_ends, how='left', on='transcript_id')
        
    ### creating map file ###

    # get gene id / name and transcript name from original gtf
    gtf_df = gtf_df.df
    gtf_df = gtf_df.loc[gtf_df.Feature == 'transcript']
    if 'gene_name' not in gtf_df.columns:
        gtf_df['gene_name'] = gtf_df.transcript_name.str.split('-', n=1, expand=True)[0]
    gtf_df = gtf_df[['gene_id', 'gene_name',
                 'transcript_id', 'transcript_name']]
    df = df.merge(gtf_df, how='left', on='transcript_id')

    # create triplets and rename old ids
    df.rename({'transcript_id': 'original_transcript_id',
               'transcript_name': 'original_transcript_name'},
               axis=1, inplace=True)
    df['transcript_triplet'] = '['+df.tss.astype(str)+','+\
                                   df.ic.astype(str)+','+\
                                   df.tes.astype(str)+']'
    df['transcript_id'] = df['gene_id']+' '+df.transcript_triplet
    df['transcript_name'] = df['gene_name']+' '+df.transcript_triplet

    return df





In [9]:
def convert_transcriptome_command(gtf, h5, o):
    convert_transcriptome(gtf, h5, o)
    
def convert_transcriptome(gtf, h5, o):
    
    # read in / format existing reference
    ic, tss, tes, _ = read_h5(h5, as_pyranges=False)
    ic = split_cerberus_id(ic, 'ic')
    tss = split_cerberus_id(tss, 'tss')
    tes = split_cerberus_id(tes, 'tes')
    tss = pr.PyRanges(tss)
    tes = pr.PyRanges(tes)
    
    # read in transcriptome to convert to cerberus
    gtf_df = pr.read_gtf(gtf).df
    gtf_df['gene_id'] = gtf_df.gene_id.str.split('.', expand=True)[0]
    gtf_df = pr.PyRanges(gtf_df)
    
    df = assign_triplets(gtf_df, tss, ic, tes)
    
    # write h5 file
    tss = tss.df
    tes = tes.df
    write_h5(ic, tss, tes, h5, m=df)

In [10]:
h5 = '/Users/fairliereese/mortazavi_lab/data/rnawg/lr_bulk/cerberus/cerberus_ref.h5'
gtf = '/Users/fairliereese/mortazavi_lab/data/rnawg/lr_bulk/talon/human_known_nic_nnc_talon.gtf'
# gtf_df, tss, ic, tes = convert_transcriptome_command(gtf, h5)
df, ic, tss, tes = convert_transcriptome(gtf, h5)



Processed 0 / 2025 genes
Processed 100 / 2025 genes
Processed 200 / 2025 genes
Processed 300 / 2025 genes
Processed 400 / 2025 genes
Processed 500 / 2025 genes
Processed 600 / 2025 genes
Processed 700 / 2025 genes
Processed 800 / 2025 genes
Processed 900 / 2025 genes
Processed 1000 / 2025 genes
Processed 1100 / 2025 genes
Processed 1200 / 2025 genes
Processed 1300 / 2025 genes
Processed 1400 / 2025 genes
Processed 1500 / 2025 genes
Processed 1600 / 2025 genes
Processed 1700 / 2025 genes
Processed 1800 / 2025 genes
Processed 1900 / 2025 genes
Processed 2000 / 2025 genes
Processed 0 / 1423 genes
Processed 100 / 1423 genes
Processed 200 / 1423 genes
Processed 300 / 1423 genes
Processed 400 / 1423 genes
Processed 500 / 1423 genes
Processed 600 / 1423 genes
Processed 700 / 1423 genes
Processed 800 / 1423 genes
Processed 900 / 1423 genes
Processed 1000 / 1423 genes
Processed 1100 / 1423 genes
Processed 1200 / 1423 genes
Processed 1300 / 1423 genes
Processed 1400 / 1423 genes


In [16]:
fname = 'talon.h5'
# tss = tss.df
# tes = tes.df

write_h5(ic, tss, tes, fname, m=df)

In [8]:
df.head()

,original_transcript_id,ic,ic_id,tss_id,tss,tes_id,tes,gene_id,gene_name,original_transcript_name,transcript_triplet,transcript_id,transcript_name
0,ENCODEHT000206942,1,ENCODEHG000058846_1,ENCODEHG000058846_1,1,ENCODEHG000058846_1,1,ENCODEHG000058846,ENCODEHG000058846,ENCODEHT000206942,"[1,1,1]","ENCODEHG000058846 [1,1,1]","ENCODEHG000058846 [1,1,1]"
1,ENCODEHT000206867,4,ENCODEHG000058837_4,ENCODEHG000058837_2,2,ENCODEHG000058837_1,1,ENCODEHG000058837,ENCODEHG000058837,ENCODEHT000206867,"[2,4,1]","ENCODEHG000058837 [2,4,1]","ENCODEHG000058837 [2,4,1]"
2,ENCODEHT000206868,2,ENCODEHG000058837_2,ENCODEHG000058837_2,2,ENCODEHG000058837_1,1,ENCODEHG000058837,ENCODEHG000058837,ENCODEHT000206868,"[2,2,1]","ENCODEHG000058837 [2,2,1]","ENCODEHG000058837 [2,2,1]"
3,ENCODEHT000206870,3,ENCODEHG000058837_3,ENCODEHG000058837_2,2,ENCODEHG000058837_1,1,ENCODEHG000058837,ENCODEHG000058837,ENCODEHT000206870,"[2,3,1]","ENCODEHG000058837 [2,3,1]","ENCODEHG000058837 [2,3,1]"
4,ENCODEHT000206886,1,ENCODEHG000058837_1,ENCODEHG000058837_1,1,ENCODEHG000058837_1,1,ENCODEHG000058837,ENCODEHG000058837,ENCODEHT000206886,"[1,1,1]","ENCODEHG000058837 [1,1,1]","ENCODEHG000058837 [1,1,1]"


In [86]:
### creating map file ###

# get gene id / name and transcript name from original gtf
gtf_df = gtf_df.df
gtf_df = gtf_df.loc[gtf_df.Feature == 'transcript']
if 'gene_name' not in gtf_df.columns:
    gtf_df['gene_name'] = gtf_df.transcript_name.str.split('-', n=1, expand=True)[0]
gtf_df = gtf_df[['gene_id', 'gene_name',
             'transcript_id', 'transcript_name']]
df = df.merge(gtf_df, how='left', on='transcript_id')

# create triplets and rename old ids
df.rename({'transcript_id': 'original_transcript_id',
           'transcript_name': 'original_transcript_name'},
           axis=1, inplace=True)
df['transcript_triplet'] = '['+df.tss.astype(str)+','+\
                               df.ic.astype(str)+','+\
                               df.tes.astype(str)+']'
df['transcript_id'] = df['gene_id']+' '+df.transcript_triplet
df['transcript_name'] = df['gene_name']+' '+df.transcript_triplet

In [88]:
df.tail()

,original_transcript_id,ic,ic_id,tss_id,tss,tes_id,tes,gene_id,gene_name,original_transcript_name,transcript_triplet,transcript_id,transcript_name
132936,ENST00000612565.1,1,ENSG00000278384_1,ENSG00000278384_1,1,ENSG00000278384_1,1,ENSG00000278384,AL354822.1,AL354822.1-201,"[1, 1, 1]","ENSG00000278384 [1, 1, 1]","AL354822.1 [1, 1, 1]"
132937,ENCODEHT000218666,2,ENSG00000273748_2,ENSG00000273748_2,2,ENSG00000273748_2,2,ENSG00000273748,AL592183.1,ENCODEHT000218666,"[2, 2, 2]","ENSG00000273748 [2, 2, 2]","AL592183.1 [2, 2, 2]"
132938,ENCODEHT000218669,3,ENSG00000273748_3,ENSG00000273748_2,2,ENSG00000273748_2,2,ENSG00000273748,AL592183.1,ENCODEHT000218669,"[2, 3, 2]","ENSG00000273748 [2, 3, 2]","AL592183.1 [2, 3, 2]"
132939,ENCODEHT001163615,1,ENCODEHG000060713_1,ENCODEHG000060713_1,1,ENCODEHG000060713_1,1,ENCODEHG000060713,ENCODEHG000060713,ENCODEHT001163615,"[1, 1, 1]","ENCODEHG000060713 [1, 1, 1]","ENCODEHG000060713 [1, 1, 1]"
132940,ENST00000616830.1,1,ENSG00000278625_1,ENSG00000278625_1,1,ENSG00000278625_1,1,ENSG00000278625,RF00026,RF00026.88-201,"[1, 1, 1]","ENSG00000278625 [1, 1, 1]","RF00026 [1, 1, 1]"


In [64]:
print(len(ends.index))
print(len(ends.loc[ends.gene_id != ends.gene_id_b].index))


93818
1871


In [67]:
fix_ends = ends.loc[ends.gene_id != ends.gene_id_b]
fix_ends = fix_ends[['Chromosome', 'Start', 'End', 'Strand',
                     'gene_id']]
print(len(fix_ends.gene_id.unique().tolist()))
ends = ends.loc[ends.gene_id == ends.gene_id_b]

1423


In [70]:
# fix the ends with mismatching gene ids - this part can be slow :(
for i, gid in enumerate(fix_ends.gene_id.unique().tolist()):
    gene_ends = fix_ends.loc[fix_ends.gene_id == gid].copy(deep=True)
    gene_ends = pr.PyRanges(gene_ends)
    gene_refs = ref.df.loc[ref.df.gene_id == gid].copy(deep=True)
    gene_refs = pr.PyRanges(gene_refs)
    gene_ends = gene_ends.nearest(ref, 
                                  strandedness=None)
    gene_ends = gene_ends.df
    ends = pd.concat([ends, gene_ends])
    
    if i % 100 == 0:
        print('Processed {} / {} genes'.format(i, len(fix_ends.gene_id.unique().tolist())))
    

Processed 0 / 1423 genes
Processed 100 / 1423 genes
Processed 200 / 1423 genes
Processed 300 / 1423 genes
Processed 400 / 1423 genes
Processed 500 / 1423 genes
Processed 600 / 1423 genes
Processed 700 / 1423 genes
Processed 800 / 1423 genes
Processed 900 / 1423 genes
Processed 1000 / 1423 genes
Processed 1100 / 1423 genes
Processed 1200 / 1423 genes
Processed 1300 / 1423 genes
Processed 1400 / 1423 genes


In [71]:
print(len(ends.index))

93818


In [75]:
t_ends = t_ends.merge(ends, how='left',
                      on=['Chromosome', 'Start', 'End', 'Strand', 'gene_id'])


In [80]:
# formatting
t_ends.rename({'Name': '{}_id'.format(mode)}, axis=1, inplace=True)
t_ends = t_ends[['transcript_id', '{}_id'.format(mode), mode]]

# merge with ic ids
df = df.merge(t_ends, how='left', on='transcript_id')

# t_ends.head()

In [81]:
df

,transcript_id,ic,ic_id,tes_id,tes
0,ENCODEHT000206942,1,ENCODEHG000058846_1,ENCODEHG000058846_1,1
1,ENCODEHT000206867,4,ENCODEHG000058837_4,ENCODEHG000058837_1,1
2,ENCODEHT000206868,2,ENCODEHG000058837_2,ENCODEHG000058837_1,1
3,ENCODEHT000206870,3,ENCODEHG000058837_3,ENCODEHG000058837_1,1
4,ENCODEHT000206886,1,ENCODEHG000058837_1,ENCODEHG000058837_1,1
...,...,...,...,...,...
132936,ENST00000612565.1,1,ENSG00000278384_1,ENSG00000278384_1,1
132937,ENCODEHT000218666,2,ENSG00000273748_2,ENSG00000273748_2,2
132938,ENCODEHT000218669,3,ENSG00000273748_3,ENSG00000273748_2,2
132939,ENCODEHT001163615,1,ENCODEHG000060713_1,ENCODEHG000060713_1,1


In [78]:
t_ends.head()
len(t_ends.index)

132941

In [18]:
gtf_df = gtf_df.df
gtf_df['gene_id'] = gtf_df.gene_id.str.split('.', n=1, expand=True)[0]
gtf_df = pr.PyRanges(gtf_df)

In [19]:
m_df = get_ic(gtf_df)
# print(m_df.head())
m_df.rename({'ic': 'Coordinates'}, axis=1, inplace=True)
m_df = m_df.merge(ic, how='left',
                  on=['Chromosome', 'Strand', 
                      'Coordinates', 'gene_id'])

# # formatting
# m_df.rename({'Name': 'ic_id'}, axis=1, inplace=True)
# m_df = m_df[['transcript_id', 'ic_id']]

In [20]:
m_df.head()

,Chromosome,Strand,transcript_id,gene_id,Coordinates,Name,source,ic
0,SIRV1,+,ENCODEHT000206942,ENCODEHG000058846,10791-10882-11057-11434,ENCODEHG000058846_1,talon,1
1,SIRV1,-,ENCODEHT000206867,ENCODEHG000058837,10647-10366-10282-7814-7552-6813-6560-6473-633...,ENCODEHG000058837_4,talon,4
2,SIRV1,-,ENCODEHT000206868,ENCODEHG000058837,10444-10366-10282-7814-7552-6813-6560-6473-633...,ENCODEHG000058837_2,talon,2
3,SIRV1,-,ENCODEHT000206870,ENCODEHG000058837,10553-7808-7552-1484,ENCODEHG000058837_3,talon,3
4,SIRV1,-,ENCODEHT000206886,ENCODEHG000058837,10282-7814-7552-6813-6337-1484,ENCODEHG000058837_1,talon,1


In [2]:
gtf_to_ics('/Users/fairliereese/mortazavi_lab/data/rnawg/lr_bulk/talon/human_known_nic_nnc_talon.gtf', o='test.tsv')

  Chromosome Strand              Coordinates            gene_id  \
0      SIRV1      +  10791-10882-11057-11434  ENCODEHG000058846   

       transcript_id  MANE_Select  basic_set  appris_principal  ic_num  \
0  ENCODEHT000206942        False      False               NaN       1   

                  Name  
0  ENCODEHG000058846_1  


In [21]:
h5 = '/Users/fairliereese/mortazavi_lab/data/rnawg/lr_bulk/cerberus/cerberus_ref.h5'
ic, tss, tes, _ = read_h5(h5)
write_h5(ic, tss, tes, h5)
ic, tss, tes, _ = read_h5(h5)

In [25]:
print(ic.loc[ic.Coordinates=='10791-10882-11057-11434'])
print(len(ic.loc[ic.Name.str.contains('ENSG')]))
print(len(ic.loc[ic.Name.str.contains('ENCODE')]))
print(len(ic.loc[~(ic.Name.str.contains('ENSG'))&~(ic.Name.str.contains('ENCODE'))]))
ic.loc[~(ic.Name.str.contains('ENSG'))&~(ic.Name.str.contains('ENCODE'))].head()
# uhhhh I don't think that there are any sirv /ercc / long sirv genes in our talon stuff oops


       Chromosome Strand              Coordinates                 Name source
241882      SIRV1      +  10791-10882-11057-11434             SIRV1B_1    v29
0           SIRV1      +  10791-10882-11057-11434  ENCODEHG000058846_1  talon
283060
441
175


,Chromosome,Strand,Coordinates,Name,source
241790,ERCC-00002,+,1045,ERCC-00002A_1,v29
241791,ERCC-00003,+,1007,ERCC-00003A_1,v29
241792,ERCC-00004,+,507,ERCC-00004A_1,v29
241793,ERCC-00009,+,968,ERCC-00009A_1,v29
241794,ERCC-00012,+,978,ERCC-00012A_1,v29


In [12]:
# ?write_h5

Signature: write_h5(ic, tss, tes, oname, m=None)
Docstring:
Write a cerberus transcriptome as an h5df file

Parameters:
    ic (pandas DataFrame): DataFrame of intron chains
    tss (pandas DataFrame): DataFrame in bed format of tsss
    tes (pandas DataFrame): DataFrame in bed format of tess
    oname (str): Output file name ending in '.h5'
    m (pandas DataFrame): DataFrame of map file
File:      ~/Documents/programming/mortazavi_lab/bin/cerberus/cerberus/cerberus.py
Type:      function
